<a href="https://colab.research.google.com/github/lcontrerasroa/meef/blob/main/ipatrace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cellule 1 — Environnement propre pour notre app (Colab)

# 0) On vire les squatteurs qui imposent des versions de websockets dont on n'a pas besoin
!pip -q uninstall -y yfinance dataproc-spark-connect google-genai google-adk || true

# 1) On s'assure qu'il n'y a pas d'ancienne version conflictuelle
!pip -q uninstall -y websockets gradio-client gradio || true

# 2) On installe Gradio stable + dépendances utiles
#    Gradio s'occupe de tirer la bonne version de gradio-client et de websockets pour lui.
!pip -q install "gradio==4.44.0" fonttools svgpathtools pillow numpy

# 3) Petit check des versions installées
import pkg_resources
def pv(name):
    try:
        return pkg_resources.get_distribution(name).version
    except:
        return "non installé"

print("gradio        =", pv("gradio"))
print("gradio-client =", pv("gradio-client"))
print("websockets    =", pv("websockets"))
print("fonttools     =", pv("fonttools"))
print("svgpathtools  =", pv("svgpathtools"))
print("pillow        =", pv("pillow"))
print("numpy         =", pv("numpy"))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
SYMBOLS_BASE = """\
a
æ
ɑ
ɒ
ɔ
e
ə
ɜ
ʌ
i
ɪ
u
ʊ
p
b
t
d
k
g
f
v
θ
ð
s
z
ʃ
ʒ
h
m
n
ŋ
l
ɫ
r
ɹ
j
w
ɾ
ʔ
ˈ
ˌ
ː
"""

DIACRITICS = """\
̩
ʰ
̃
̚
̠
̟
̈
̹
͡
"""

def parse_list(block:str):
    return [line.strip() for line in block.splitlines() if line.strip()]

symbols_base = parse_list(SYMBOLS_BASE)
diacritics = parse_list(DIACRITICS)

print("Base:", symbols_base)
print("Diacritiques:", diacritics)

Base: ['a', 'æ', 'ɑ', 'ɒ', 'ɔ', 'e', 'ə', 'ɜ', 'ʌ', 'i', 'ɪ', 'u', 'ʊ', 'p', 'b', 't', 'd', 'k', 'g', 'f', 'v', 'θ', 'ð', 's', 'z', 'ʃ', 'ʒ', 'h', 'm', 'n', 'ŋ', 'l', 'ɫ', 'r', 'ɹ', 'j', 'w', 'ɾ', 'ʔ', 'ˈ', 'ˌ', 'ː']
Diacritiques: ['̩', 'ʰ', '̃', '̚', '̠', '̟', '̈', '̹', '͡']


In [ ]:
import urllib.request, zipfile, os, shutil

# Choisis ta police ici : 'doulos', 'charis' ou 'gentium'
FONT_FAMILY = "doulos"  # <-- change ici si tu veux

urls = {
    "doulos": "https://software.sil.org/downloads/r/doulos/DoulosSIL-6.200.zip",
    "charis": "https://software.sil.org/downloads/r/charis/CharisSIL-6.200.zip",
    "gentium": "https://software.sil.org/downloads/r/gentium/GentiumPlus-6.200.zip"
}

assert FONT_FAMILY in urls, "Police inconnue, choisis 'doulos', 'charis' ou 'gentium'."

url = urls[FONT_FAMILY]
zip_path = f"{FONT_FAMILY}.zip"
font_dir = f"{FONT_FAMILY}_font"

# Télécharger et extraire
print(f"Téléchargement de {url} ...")
urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(font_dir)

# Chercher le premier .ttf dans le dossier
ttfs = [os.path.join(font_dir, f) for f in os.listdir(font_dir) if f.lower().endswith(".ttf")]
assert ttfs, "Aucun fichier .ttf trouvé dans le zip."
FONT_PATH = ttfs[0]
print("Police prête :", FONT_PATH)

Téléchargement de https://software.sil.org/downloads/r/doulos/DoulosSIL-6.200.zip ...


AssertionError: Aucun fichier .ttf trouvé dans le zip.

In [ ]:
def build_cmap(tt):
    cmap = {}
    for table in tt["cmap"].tables:
        if table.isUnicode():
            cmap.update(table.cmap)
    return cmap

def char_to_glyph(tt, ch, cmap):
    cp = ord(ch)
    if cp in cmap:
        return tt.getGlyphName(cmap[cp])
    # fallback basique: NFD et première composante
    decomp = unicodedata.normalize("NFD", ch)
    if decomp and ord(decomp[0]) in cmap:
        return tt.getGlyphName(cmap[ord(decomp[0])])
    return None

def glyph_path_d(tt, glyph_name):
    glyph_set = tt.getGlyphSet()
    if glyph_name not in glyph_set:
        return None
    pen = SVGPathPen(glyph_set)
    glyph_set[glyph_name].draw(pen)
    return pen.getCommands()

def write_svg(d_path, upem, adv_width, out_path, viewbox_em=1000, stroke_px=10, padding_pct=8):
    pad = viewbox_em * (padding_pct / 100.0)
    vb_w = viewbox_em + 2 * pad
    vb_h = viewbox_em + 2 * pad

    scale_x = viewbox_em / max(adv_width, 1)
    scale_y = viewbox_em / max(upem, 1)

    translate_x = pad + (vb_w - 2 * pad - viewbox_em) / 2.0
    translate_y = pad + viewbox_em

    svg = f'''<?xml version="1.0" encoding="UTF-8"?>
<svg width="{int(vb_w)}" height="{int(vb_h)}" viewBox="0 0 {vb_w} {vb_h}" xmlns="http://www.w3.org/2000/svg">
  <rect x="0" y="0" width="{vb_w}" height="{vb_h}" fill="white"/>
  <g transform="translate({translate_x:.3f},{translate_y:.3f}) scale({scale_x:.5f},{-scale_y:.5f})">
    <path d="{d_path}" fill="none" stroke="#000" stroke-width="{stroke_px}" stroke-linecap="round" stroke-linejoin="round"/>
  </g>
</svg>'''
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(svg)

def export_svgs(font_path, chars, outdir="svgs_base", stroke=10, padding=6, size=1000):
    tt = TTFont(font_path)
    upem = tt["head"].unitsPerEm
    hmtx = tt["hmtx"]
    cmap = build_cmap(tt)
    os.makedirs(outdir, exist_ok=True)
    produced = []

    for ch in chars:
        gname = char_to_glyph(tt, ch, cmap)
        if not gname:
            print(f"[!] Pas de glyphe pour: {repr(ch)} U+{ord(ch):04X}")
            continue
        d = glyph_path_d(tt, gname)
        if not d:
            print(f"[!] Glyphe vide: {ch} -> {gname}")
            continue
        adv_width, _lsb = hmtx[gname]

        # nom de fichier safe
        fname = ch
        if ch in r'\/:*?"<>| ' or not ch.isprintable():
            fname = f"u{ord(ch):04X}"
        out_path = os.path.join(outdir, f"{fname}.svg")

        write_svg(d, upem, adv_width, out_path, viewbox_em=size, stroke_px=stroke, padding_pct=padding)
        produced.append((ch, out_path))
        print(f"[OK] {ch} -> {out_path}")
    return produced

In [ ]:
svgs_base = export_svgs(FONT_PATH, symbols_base, outdir="svgs_base", stroke=10, padding=6, size=1000)
svgs_diac = export_svgs(FONT_PATH, diacritics,  outdir="svgs_diacritics", stroke=10, padding=6, size=1000)

len(svgs_base), len(svgs_diac)

In [ ]:
def rasterize_svg_path(svg_path, size=400, line_px=3):
    # parse <path d="...">
    with open(svg_path, "r", encoding="utf-8") as f:
        svg = f.read()
    m = re.search(r'd="([^"]+)"', svg)
    if not m:
        # fallback: lire tout en bitmap blanc
        return Image.new("L", (size, size), 255)
    d = m.group(1)

    # lire aussi la viewBox pour l’échelle
    vb = re.search(r'viewBox="([\d\.\s]+)"', svg)
    if vb:
        x0,y0,w,h = map(float, vb.group(1).split())
    else:
        x0,y0,w,h = 0,0,1000,1000

    # échantillonnage du chemin via svgpathtools
    # on va relire l’ensemble des paths pour sécurité
    paths, _ = svg2paths(svg_path)
    img = Image.new("L", (size, size), 255)
    drw = ImageDraw.Draw(img)

    for p in paths:
        # nombre de pas: proportionnel à la longueur
        total_len = sum(seg.length() for seg in p)
        steps = max(100, int(total_len / (w/size) * 2))
        pts = []
        for i in range(steps+1):
            t = i/steps
            z = p.point(t)
            x = (z.real - x0) * (size / w)
            y = (z.imag - y0) * (size / h)
            pts.append((x,y))
        # tracer en noir (0)
        drw.line(pts, fill=0, width=line_px, joint="curve")
    return img

In [ ]:
PREVIEW_SIZE = 144

gallery_items = []
symbol_to_svg = {}
for ch, path in svgs_base:
    thumb = rasterize_svg_path(path, size=PREVIEW_SIZE, line_px=3)
    # bord fin pour la galerie
    thumb = ImageOps.expand(thumb.convert("RGB"), border=2, fill=(230,230,230))
    gallery_items.append(thumb)
    symbol_to_svg[ch] = path

# ordre stable
symbol_list = [ch for ch,_ in svgs_base]
len(gallery_items), len(symbol_list)

In [ ]:
CANVAS_SIZE = 400
STENCIL_ALPHA = 160  # 0..255
DILATION = 2         # dilatation légère du stencil pour tolérance

def get_stencil(ch):
    svg_path = symbol_to_svg.get(ch)
    if not svg_path:
        return None
    base = rasterize_svg_path(svg_path, size=CANVAS_SIZE, line_px=4)
    # convertir en RGBA + alpha pour superposition
    rgba = Image.new("RGBA", (CANVAS_SIZE, CANVAS_SIZE), (255,255,255,0))
    # pixels noirs = trait -> dessiner en gris
    arr = np.array(base)
    mask = (arr < 128).astype(np.uint8)*255
    gray = Image.new("RGBA", (CANVAS_SIZE, CANVAS_SIZE), (0,0,0,STENCIL_ALPHA))
    rgba.paste(gray, mask=Image.fromarray(mask))
    return rgba, mask  # mask binaire du modèle

def sketch_to_mask(sketch_img):
    if sketch_img is None:
        return np.zeros((CANVAS_SIZE, CANVAS_SIZE), dtype=np.uint8)
    if isinstance(sketch_img, dict) and "image" in sketch_img:
        sketch_img = sketch_img["image"]
    im = Image.fromarray(sketch_img).convert("L")
    # le sketchpad met fond blanc, trait noir: seuil
    m = np.array(im) < 200
    return (m.astype(np.uint8)*255)

def dilate(mask, iters=1):
    # dilation bête en 4-connexité
    m = mask.copy().astype(np.uint8)
    for _ in range(iters):
        up = np.pad(m, ((1,0),(0,0)), constant_values=0)[:-1,:]
        down = np.pad(m, ((0,1),(0,0)), constant_values=0)[1:,:]
        left = np.pad(m, ((0,0),(1,0)), constant_values=0)[:, :-1]
        right= np.pad(m, ((0,0),(0,1)), constant_values=0)[:, 1:]
        m = np.maximum.reduce([m, up, down, left, right])
    return m

def score_masks(user_mask, model_mask):
    # Dice coefficient sur pixels dessinés
    u = (user_mask > 0).astype(np.uint8)
    m = (model_mask > 0).astype(np.uint8)
    # tolérance: on dilate légèrement le modèle
    m_d = dilate(m*255, iters=DILATION) > 0
    inter = np.logical_and(u>0, m_d>0).sum()
    denom = (u>0).sum() + (m>0).sum()
    if denom == 0:
        return 0.0
    return 2*inter/denom

# état global simple
DEFAULT_SYMBOL = symbol_list[0] if symbol_list else "a"

def select_from_gallery(evt: gr.SelectData, current_symbol):
    # evt.index -> index du clic
    idx = evt.index
    if isinstance(idx, (tuple, list)):  # Gallery peut renvoyer (row, col); on s'en sort
        idx = idx[0]*999 + idx[1]  # fallback absurde; Gradio renvoie normalement un index plat
    try:
        ch = symbol_list[idx]
    except:
        ch = current_symbol or DEFAULT_SYMBOL
    rgba, mask = get_stencil(ch)
    return ch, rgba

def change_symbol(ch):
    rgba, mask = get_stencil(ch)
    return rgba

def evaluate(sketch_img, ch):
    rgba, model_mask = get_stencil(ch)
    user_mask = sketch_to_mask(sketch_img)
    sc = score_masks(user_mask, model_mask)
    msg = "Très bien." if sc >= 0.85 else ("Correct." if sc >= 0.65 else "À retravailler.")
    return f"Score: {sc*100:.1f}% — {msg}"

with gr.Blocks(title="Tracer l’API") as demo:
    gr.Markdown("## Tracer les symboles de l’API — grille, pochoir, score")
    with gr.Row():
        with gr.Column(scale=1):
            gallery = gr.Gallery(gallery_items, label="Choisis un caractère",
                                 columns=6, rows=4, height=480, allow_preview=False)
            symbol = gr.Dropdown(choices=symbol_list, value=DEFAULT_SYMBOL, label="Symbole")
        with gr.Column(scale=2):
            stencil = gr.Image(label="Modèle (stencil)", interactive=False, height=CANVAS_SIZE, width=CANVAS_SIZE)
            sketch = gr.Sketchpad(label="À toi de tracer", shape=(CANVAS_SIZE, CANVAS_SIZE))
            with gr.Row():
                btn_eval = gr.Button("Évaluer")
                btn_clear = gr.Button("Effacer")
            out = gr.Markdown()

    # init
    stencil.value = change_symbol(DEFAULT_SYMBOL)

    # events
    gallery.select(fn=select_from_gallery, inputs=[symbol], outputs=[symbol, stencil])
    symbol.change(fn=change_symbol, inputs=symbol, outputs=stencil)
    btn_eval.click(fn=evaluate, inputs=[sketch, symbol], outputs=out)
    btn_clear.click(lambda: None, None, sketch)

demo.launch(share=True)

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9167e4c61f36f65edd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
               ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/routing.py", line 387, in app
    raw_response = await run_endpoint_function(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/routing.py", line 290, in run_endpoint_function
    return await run_in_threadpool(dependant.call, **values)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/starlette/concurrency.py", line 38, in run_in_threadpool
    return await anyio.to_thread.run_sync(func)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/_backends

In [ ]:
# Dans une cellule Colab
!zip -r ipa_svgs.zip svgs_base svgs_diacritics 2>/dev/null